In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.1 MB/s eta 0:00:00


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd

# Example file path
file_path = '/content/drive/MyDrive/projects/job_skill_extraction/postings.csv'

df = pd.read_csv(file_path)
df.shape, df.columns


((123849, 31),
 Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
        'pay_period', 'location', 'company_id', 'views', 'med_salary',
        'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
        'remote_allowed', 'job_posting_url', 'application_url',
        'application_type', 'expiry', 'closed_time',
        'formatted_experience_level', 'skills_desc', 'listed_time',
        'posting_domain', 'sponsored', 'work_type', 'currency',
        'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
       dtype='object'))

In [ ]:

# Preview
df.shape, df.columns.tolist()


((123849, 31),
 ['job_id',
  'company_name',
  'title',
  'description',
  'max_salary',
  'pay_period',
  'location',
  'company_id',
  'views',
  'med_salary',
  'min_salary',
  'formatted_work_type',
  'applies',
  'original_listed_time',
  'remote_allowed',
  'job_posting_url',
  'application_url',
  'application_type',
  'expiry',
  'closed_time',
  'formatted_experience_level',
  'skills_desc',
  'listed_time',
  'posting_domain',
  'sponsored',
  'work_type',
  'currency',
  'compensation_type',
  'normalized_salary',
  'zip_code',
  'fips'])

In [ ]:
df['description'].sample(5).values


array(["Hybrid Details: Boston, MA\n\nDuration: 12 months to start\n\nJob Description\n\nOur client is seeking a candidate who can assist and enhance development efforts within the IT - Application Operations Group and build/support enterprise-wide applications using the Microsoft Power Platform, SharePoint Online, Adobe eSign and related technologies. This role requires the candidate to adhere to IT policies, governance, and act as technical advisor/liaison in various business interactions, provide guidance to internal development teams, vendors/partners. The candidate also needs to support end-to-end life cycle of current, future, and in-flight Power platform applications that integrate with M365 Services and other third-party platforms.\n\nJob Requirements\n\nSix (6) plus years of solid hands-on development experience working within Microsoft 365 SharePoint Online platform as well as SharePoint On-Premises Servers (2013, 2019) and cloud development platforms including Power Platform

In [ ]:
import re
from bs4 import BeautifulSoup


In [ ]:
def clean_description(text):
    if pd.isnull(text):
        return ""

    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove bullet points, special characters (keep words, numbers, punctuation)
    text = re.sub(r"[\r\n•●▪★✔➡➤➔]", " ", text)  # remove bullets and newlines
    text = re.sub(r"[^\w\s.,$%/-]", "", text)      # remove weird symbols

    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text)

    # Lowercase everything
    return text.strip().lower()


In [ ]:
df['description_cleaned'] = df['description'].apply(clean_description)


<ipython-input-8-add4932b1d8a>:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
df[['description', 'description_cleaned']].sample(5)


,description,description_cleaned
96277,Job#: 2022793\n\nJob Description:\n\nApex Syst...,"job 2022793 job description apex systems, a wo..."
118526,Job Title- Front-End Developer - Analyst/Devel...,job title- front-end developer - analyst/devel...
41112,Do you thrive in the dynamic world of sales an...,do you thrive in the dynamic world of sales an...
81322,"Job Description\n\nBenchmark, a brand partner ...","job description benchmark, a brand partner of ..."
46969,Ingersoll Rand is committed to achieving workf...,ingersoll rand is committed to achieving workf...


# **Create a Skill Dictionary**

In [ ]:
# You can expand this list as needed
skill_list = [
    'python', 'java', 'c++', 'javascript', 'typescript', 'html', 'css',
    'sql', 'nosql', 'mongodb', 'postgresql', 'mysql',
    'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'linux',
    'react', 'angular', 'vue', 'node.js', 'django', 'flask', 'spring',
    'git', 'jenkins', 'jira', 'excel', 'powerpoint',
    'machine learning', 'deep learning', 'nlp', 'data analysis', 'data science',
    'communication', 'leadership', 'problem solving', 'teamwork', 'project management'
]

# Convert to lowercase for matching
skill_set = set([skill.lower() for skill in skill_list])


In [ ]:
def extract_skills(text):
    if not isinstance(text, str):
        return []

    found = []
    for skill in skill_set:
        if skill in text:
            found.append(skill)

    return list(set(found))  # Remove duplicates


In [ ]:
df['extracted_skills'] = df['description_cleaned'].apply(extract_skills)


In [ ]:
df[['title', 'description_cleaned', 'extracted_skills']].sample(5)


,title,description_cleaned,extracted_skills
52007,Salesforce Developer ( Marketing Cloud),note need local candidates and 10 years experi...,"[git, communication]"
44163,#13476 Manual Tester,description are you interested in working with...,"[excel, leadership, communication, jira]"
50260,Patient Service Representative,summary the patient service associate is respo...,[]
2311,Senior Accountant,the sr. accountant is primarily responsible fo...,"[aws, teamwork, excel, communication]"
45323,Business Analyst II,overview welcome home build your career with l...,"[aws, java, project management, powerpoint, pr..."


In [ ]:
from rapidfuzz import fuzz

# Skill mapping (synonyms → canonical form)
skill_synonyms = {
    'js': 'javascript',
    'java script': 'javascript',
    'ml': 'machine learning',
    'gcloud': 'gcp',
    'k8s': 'kubernetes',
    'postgre': 'postgresql',
    'gitlab': 'git',
    'tensorflow': 'deep learning',
    'power point': 'powerpoint',
}


In [ ]:
def fuzzy_extract_skills(text, skills=skill_set, synonyms=skill_synonyms):
    found = set()

    if not isinstance(text, str):
        return []

    # Exact skill matches
    for skill in skills:
        if skill in text:
            found.add(skill)

    # Fuzzy match synonyms
    for alt, canonical in synonyms.items():
        score = fuzz.partial_ratio(alt, text)
        if score >= 90:  # tune this threshold as needed
            found.add(canonical)

    return list(found)


In [ ]:
df['extracted_skills_fuzzy'] = df['description_cleaned'].apply(fuzzy_extract_skills)


In [ ]:
df[df['extracted_skills_fuzzy'].apply(lambda x: len(x) > 0)][['title', 'extracted_skills_fuzzy']].head(10)


,title,extracted_skills_fuzzy
1,Mental Health Therapist/Counselor,"[excel, communication]"
3,Senior Elder Law / Trusts and Estates Associat...,"[excel, communication]"
5,Economic Development and Planning Intern,"[excel, leadership, communication, data analysis]"
7,Building Engineer,"[excel, leadership, project management, commun..."
8,Respiratory Therapist,"[excel, react]"
9,Worship Leader,"[excel, leadership]"
10,Inside Customer Service Associate,"[excel, communication, machine learning]"
11,Project Architect,"[excel, leadership, git]"
12,Appalachian Highlands Women's Business Center,"[excel, leadership, project management, commun..."
14,Senior Product Marketing Manager,"[excel, communication, git, machine learning]"


# **Extract Unknown Skills using spaCy**

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
# Random sample of 2000 rows (recommended)
df_sample = df.sample(2000, random_state=42).copy()

# OR just take first 1000 rows
# df_sample = df.head(1000).copy()


In [ ]:
from tqdm.notebook import tqdm

def extract_noun_chunks_batch(texts):
    noun_chunks = []
    for doc in tqdm(nlp.pipe(texts, disable=["ner", "textcat"]), total=len(texts)):
        chunks = [chunk.text.strip().lower() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
        noun_chunks.append(list(set(chunks)))
    return noun_chunks

# Apply only to sample
df_sample['noun_chunks'] = extract_noun_chunks_batch(df_sample['description_cleaned'])


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
df_sample[['title', 'noun_chunks']].head(5)


,title,noun_chunks
73989,Senior Automation Engineer - Power Systems,"[a team, radiation, reports, actual system per..."
59308,DISH Installation Technician - Field,"[a technician, customers homes, a row, hughes,..."
44663,Order Builder,"[compliance, the staging areacheck, quantity, ..."
81954,"Mountain Multimedia Journalist, KMGH","[quality, pet discount program, the ideal cand..."
113151,Licensed Practical Nurse (LPN),"[one careflexible scheduling, these identities..."


In [ ]:
# Define words/phrases you want to filter out
stop_chunks = {
    'we', 'they', 'you', 'i', 'this', 'that', 'who', 'our team', 'our lpns',
    'the principle', 'our highest priority', 'your work-life balance', 'the culture',
    'sexual orientation', 'race', 'color', 'disability', 'gender identity',
    'military status', 'citizenship status', 'age', 'all employees', 'sex',
    'other marginalized communities', 'reasonable accommodations',
    'a non-discriminatory basis', 'equal opportunity employers',
    'our nurses', 'our clients', 'our talent network', 'clients', 'members', 'people',
    '.', 'the office', 'success', 'the job', 'our benefits'
}


In [ ]:
def filter_noun_chunks(chunk_list, stopwords=stop_chunks):
    if not isinstance(chunk_list, list):
        return []

    return [
        chunk for chunk in chunk_list
        if chunk not in stopwords and len(chunk) > 2 and not chunk.isnumeric()
    ]


In [ ]:
df_sample['noun_chunks_filtered'] = df_sample['noun_chunks'].apply(filter_noun_chunks)


In [ ]:
df_sample[['title', 'noun_chunks_filtered']].sample(5)


,title,noun_chunks_filtered
63158,Business Development Associate,"[a team, goals, revenue, your story, your spam..."
39748,Equipment & Controls Planning Engineer -24-00269,"[quality, approved budget, ensures, project ma..."
44576,"Medical Laboratory Assistant, Microbiology/Vir...","[transport, hours, receives, body fluids, dire..."
105515,Community Mortgage Bank Sales Consultant,"[quality, mortgage industry trends, compliance..."
45768,Senior Facilities Engineer,"[a team, new parents, high-volume manufacturin..."
